In [ ]:
from hydromt_sfincs import SfincsModel
from hydromt.config import configread
from hydromt.log import setuplog
from os.path import join, isfile, basename, isdir
import subprocess
import matplotlib.pyplot as plt
import numpy as np
import glob
import shutil

In [ ]:
# model and data paths
model_root = r'../../3_models/SFINCS'
base_root = join(model_root, "10_base_50m")
data_libs = [
    r'../../1_data/1_static/static_data.yml',
    r'../../1_data/2_forcing/forcing_data.yml',    
]
logger = setuplog('update', join(base_root, "hydromt.log"), log_level=10)

# NOTE: please contact the autors for an executable
fn_exe = "p:/11205283-hydromt-floodmodelling/02_models/bin/subgrid_openacc_11_rev295_16092021/sfincs.exe"


In [ ]:
# build model base layers
region = {'bbox': [34.33,-20.12,34.95,-19.30]}
res = 50 # resolution [m]
opt = configread('build_sfincs.ini', abs_path=True)
kwargs = opt.pop('global',{})
mod = SfincsModel(root=base_root, data_libs=data_libs, logger=logger, **kwargs)
mod.build(region=region, res=res, opt=opt)
mod.plot_basemap()

In [ ]:
# update base model river bathymetry
base_root = join(model_root, "00_base_100m")
runs = [
    (1, {'rivdph_method': 'powlaw'}, {}),
    (2, {'rivdph_method': 'manning'}, {}),
    (3, {'rivdph_method': 'gvf'}, {}),
    (4, {'rivdph_method': 'powlaw', 'hp': 0.25}, {}),
    (5, {'rivdph_method': 'powlaw', 'hp': 0.35}, {}),
    (6, {'rivdph_method': 'powlaw', 'hc': 0.50}, {}),
    (7, {'rivdph_method': 'gvf', 'rivbank': False}, {}),
    (8, {'rivdph_method': 'powlaw', 'river_upa': 25}, {}),
    (9, {'rivdph_method': 'powlaw'}, {'lnd_man': 0.05}),
]
# base_root = join(model_root, "10_base_50m")
# runs = [
#     (11, {'rivdph_method': 'powlaw'}),
#     (12, {'rivdph_method': 'manning'}),
#     (13, {'rivdph_method': 'gvf'}),
#     (18, {'rivdph_method': 'powlaw', 'river_upa': 25}),
# ]
for i, kwargs, kwargs1 in runs:
    opt = configread(f'update_rivbathy.ini', abs_path=True)
    postfix = '_'.join([f'{k[:3]}{v}' for k,v in kwargs.items()])
    root = join(model_root, f'{i:02d}_{postfix}')
    if kwargs1:
        postfix1 = '_'.join([f'{k[:3]}{v}' for k,v in kwargs1.items()])
        root = f'{root}_{postfix1}'
    if isdir(root):
        continue
    mod1 = SfincsModel(base_root, mode='r', data_libs=data_libs, logger=logger)
    # update bathymetry with specific settings
    opt['setup_river_bathymetry'].update(kwargs)
    opt['setup_manning_roughness'].update(kwargs1)
    mod1.update(model_out=root, opt=opt, write=False)
    # set zs ini restart file
    mask = np.logical_or(mod1.staticmaps['rivmsk']==1, mod1.staticmaps['dep']<0)
    zsini = mod1.staticmaps['dep'].where(~mask, np.maximum(mod1.staticmaps['dep']+0.2, 0.5)).where(mod1.mask!=0,0)
    zsini.raster.set_nodata(0)
    mod1.set_states(zsini, 'zsini')
    mod1.config.pop('zsini',None)
    # write static maps, states and config
    mod1.write()
    mod1.plot_basemap(geoms=[])
    plt.close('all')

In [ ]:
# write forcing
runs = [r for r in glob.glob(f'../../3_models/SFINCS/*') if not isfile(r)]
for root0 in runs:
    # update forcing in subfolders
    for event in ['idai', 'eloise', 'idai_glofas', 'eloise_glofas']:
        root = join(root0, event)
        if isdir(root):
            continue
        mod1 = SfincsModel(root0, mode='r', data_libs=data_libs, logger=logger)
        opt = configread(f'update_sfincs_{event}.ini', abs_path=True)
        # glofas forcing only with default 01_ run
        if 'glofas' in basename(event):
            if not basename(root0).startswith('01'): continue
            opt['setup_q_forcing_from_grid'].update({
                'discharge_fn': 'glofas_era5', 'uparea_fn': 'glofas_uparea'
            })
        mod1.update(model_out=root, opt=opt, write=False)
        # write forcing and sfincs.inp only
        mod1.write_forcing()
        mod1.write_config(rel_path=f'../')
        mod1.plot_forcing()
        plt.close('all')
    # break

In [ ]:
from itertools import product
from hydromt_sfincs.utils import write_timeseries, write_inp
import os
## prepare compound runs

base_root = join(model_root, "01_rivpowlaw")
for event in ['idai', 'eloise']:
    mod = SfincsModel(join(base_root, event), mode='r', data_libs=data_libs)
    config = mod.config.copy()
    tref = config['tref']

    # get htide
    htot = mod.forcing['bzs'].copy()
    mod.setup_h_forcing(
        geodataset_fn = f'gtsm_{event}_tides',   # waterlevel timeseries dataset
    )
    htide = mod.forcing['bzs'].copy()
    assert np.all(htide.vector.xcoords.round(0) == htot.vector.xcoords.round(0))
    assert np.all(htide.vector.ycoords.round(0) == htot.vector.ycoords.round(0))

    # get discharge climatology
    da_dis = mod.forcing['dis'].copy()
    # mod.setup_q_forcing_from_grid(
    #     discharge_fn = 'glofas_era5',
    #     uparea_fn = 'glofas_uparea',   
    # )
    # da_dis_glofas = mod.forcing['dis'].copy()
    mod.setup_config(
        tstart = '19800101 000000',
        tend = '20200101 000000',
    )
    mod.setup_q_forcing_from_grid(
        discharge_fn = 'cmf_outflw_06min',   # TODO change to 3min version
        uparea_fn = 'cmf_uparea_06min',   
    )
    da_dis_long = mod.forcing['dis'].copy()
    # qmean = da_dis_long.groupby('time.quarter').mean().sel(quarter=int(np.mean(da_dis.time.dt.quarter)))
    # qmean = da_dis_long.groupby('time.month').mean().sel(month=int(np.mean(da_dis.time.dt.month)))
    qmean = da_dis_long.mean('time')
    fact = np.minimum(1,qmean/da_dis.mean())
    print(fact.values)
    da_disclim = da_dis*fact

    qs = {
        'disclim': da_disclim.reset_coords(drop=True).to_series().unstack(0),
        'dis': da_dis.reset_coords(drop=True).to_series().unstack(0), 
    }
    hs = {
        'htide': htide.reset_coords(drop=True).to_series().unstack(0),
        'htot': htot.reset_coords(drop=True).to_series().unstack(0),
    }
    ps = {
        'noprecip': None,
        'precip': mod.forcing['netampr'].copy(),
    }
    cmpd_runs = {
        'q': ['dis', 'htide', 'noprecip'],
        'h': ['disclim', 'htot', 'noprecip'],
        'p': ['disclim', 'htide', 'precip'],
        'base': ['disclim', 'htide', 'noprecip'],
        'qh': ['dis', 'htot', 'noprecip'],
        }
    for run, (q, h, p) in cmpd_runs.items():
        root = join(base_root, f'{event}_{run}')
        if not os.path.isdir(root):
            os.makedirs(root)
        else:
            continue
        print(f'{event}_{run}')

        shutil.copyfile(join(base_root, event, 'sfincs.src'), join(root, 'sfincs.src'))
        write_timeseries(join(root, 'sfincs.dis'), qs[q], tref)
        shutil.copyfile(join(base_root, event, 'sfincs.bnd'), join(root, 'sfincs.bnd'))
        write_timeseries(join(root, 'sfincs.bzs'), hs[h], tref)
        
        config1 = config.copy()
        if ps[p] is None:
            config1.pop('netamprfile')
        else:
            try:
                ps[p].to_netcdf(join(root, 'precip.nc'))
            except:
                pass

        write_inp(join(root, 'sfincs.inp'), config1)

        mod1 = SfincsModel(root, mode='r')
        mod1.plot_forcing()
        plt.close('all')

In [ ]:
# run models
from os.path import dirname

def check_finished(root):
    finished = False
    if isfile(join(root, 'sfincs.log')):
        with open(join(root, 'sfincs.log'), 'r') as f:
            finished = np.any(['Simulation is finished' in l for l in f.readlines()])
    return finished

runs = [dirname(fn) for fn in glob.glob(join(model_root, '*', '*', 'sfincs.inp')) if not check_finished(dirname(fn))]
n = len(runs)
print(n)
for i, root in enumerate(runs):
    print(f'{i+1:d}/{n:d}: {root}')
    with open(join(root, "sfincs.log"), 'w') as f:
        p = subprocess.Popen([fn_exe], stdout=f, cwd=root)
        %time p.wait()